# Differential expression analysis


**You have now successfully run the pipeline and checked the first quality control metrics of your fastq files. However, this is only primary analysis.**

**We would now like to understand exactly the difference between our groups of mice.** 

**Which pipeline would you use for this?**


differentialabundance

**Please paste here the command you used**

In [26]:

# for running with the lengths, the results will be more accurate, but the pipeline might take longer

'''
sample,fastq_1,fastq_2,condition,replicate,batch
CONTROL_REP1,AEG588A1_S1_L002_R1_001.fastq.gz,AEG588A1_S1_L002_R2_001.fastq.gz,control,1,A
CONTROL_REP2,AEG588A1_S1_L003_R1_001.fastq.gz,AEG588A1_S1_L003_R2_001.fastq.gz,control,2,B
CONTROL_REP3,AEG588A1_S1_L004_R1_001.fastq.gz,AEG588A1_S1_L004_R2_001.fastq.gz,control,3,A
TREATED_REP1,AEG588A2_S1_L002_R1_001.fastq.gz,AEG588A2_S1_L002_R2_001.fastq.gz,treated,1,B
TREATED_REP2,AEG588A2_S1_L003_R1_001.fastq.gz,AEG588A2_S1_L003_R2_001.fastq.gz,treated,2,A
TREATED_REP3,AEG588A2_S1_L004_R1_001.fastq.gz,AEG588A2_S1_L004_R2_001.fastq.gz,treated,3,B
'''

# with fastq paths omitted

'''
sample,condition,replicate,batch
CONTROL_REP1,control,1,A
CONTROL_REP2,control,2,B
CONTROL_REP3,control,3,A
TREATED_REP1,treated,1,B
TREATED_REP2,treated,2,A
TREATED_REP3,treated,3,B
'''

# minimal sample sheet

'''
sample,condition
CONTROL_REP1,control
CONTROL_REP2,control
CONTROL_REP3,control
TREATED_REP1,treated
TREATED_REP2,treated
TREATED_REP3,treated
'''

import os
import pandas as pd

output_name = "samplesheet.csv"

# metadata
input_file = "data/salmon.merged.gene_counts.tsv"

# convert input tsv to pandas
data = pd.read_csv(input_file, sep='\t')
#print(data)

# Headers
# FastQ files can be omitted apprently
sample_sheet = pd.DataFrame(columns=['sample', 'condition'])
sample_names = data.columns[2:] 
sample_sheet['sample'] = sample_names
sample_sheet['condition'] = sample_sheet['sample'].str.replace(r'_\d+$', '', regex=True)

sample_sheet.to_csv('sample_sheet.csv', index=False)

In [36]:
# Creating the contrasts file


'''
sample,      fastq_1,                         fastq_2,                       condition,replicate,batch
CONTROL_REP1,AEG588A1_S1_L002_R1_001.fastq.gz,AEG588A1_S1_L002_R2_001.fastq.gz,control,  1,         A
CONTROL_REP2,AEG588A1_S1_L003_R1_001.fastq.gz,AEG588A1_S1_L003_R2_001.fastq.gz,control,  2,         B
CONTROL_REP3,AEG588A1_S1_L004_R1_001.fastq.gz,AEG588A1_S1_L004_R2_001.fastq.gz,control,  3,         A
TREATED_REP1,AEG588A2_S1_L002_R1_001.fastq.gz,AEG588A2_S1_L002_R2_001.fastq.gz,treated,  1,         B
TREATED_REP2,AEG588A2_S1_L003_R1_001.fastq.gz,AEG588A2_S1_L003_R2_001.fastq.gz,treated,  2,         A
TREATED_REP3,AEG588A2_S1_L004_R1_001.fastq.gz,AEG588A2_S1_L004_R2_001.fastq.gz,treated,  3,         B
'''

#The contrasts file references the observations file to define groups of samples to compare. 
#For example, based on the sample sheet above we could define contrasts like:

'''
id,                                  variable,  reference, target,  blocking

condition_control_treated,           condition,  control,   treated,
condition_control_treated_blockrep,  condition,  control,   treated,  replicate;batch
'''

'''

id - an arbitrary identifier, will be used to name contrast-wise output files 

variable - which column from the observations information will be used to define groups

reference - the base/ reference level for the comparison. 
If features have higher values in this group than target they will generate negative fold changes
# ex: 'Placebo'

target - the target/ non-reference level for the comparison. 
If features have higher values in this group than the reference they will generate positive fold changes
# ex: 'Drug' 

'''

contrasts_sheet = pd.DataFrame(columns=['id', 'variable', 'reference', 'target'])

contrasts_sheet.loc[0] = ["a", "condition", "Sham_Sal", "Sham_oxy"] # Sham-Oxy vs. Sham-Sal
contrasts_sheet.loc[1] = ["b", "condition", "Sham_Sal", "SNI_Sal"] # SNI-Sal vs Sham-Sal,
contrasts_sheet.loc[2] = ["c", "condition", "Sham_Sal", "SNI_oxy"] # SNI-Oxy vs Sham-Sal

#contrasts_sheet.loc[3] = ["d", "condition", "SNI_Sal", "SNI_oxy"] # not used in the study 
#contrasts_sheet.loc[4] = ["e", "condition", "Sham_oxy", "SNI_oxy"] # not used in the study 

contrasts_sheet.to_csv('contrasts.csv', index=False)

In [ ]:
# running the pipeline

nextflow run nf-core/differentialabundance \
     --input sample_sheet.csv \
     --contrasts contrasts.csv \
     --matrix "data/salmon.merged.gene_counts.tsv" \
     --outdir results_diff_ana_v3  \
     -profile docker

**Explain all the parameters you set and why you set them in this way.**


```x
--input sample_sheet.csv                                        required input file
--contrasts contrasts.csv                                       required input file
--matrix "data/salmon.merged.gene_counts.tsv"   required input file (output from rnaswq)
--outdir results_diff_ana_v3                                       required output path
-profile docker                                                 recommended to use docker.


**How did the pipeline perform?**

```x

Completed at: 02-Oct-2024 11:19:42
Duration    : 4m 18s
CPU hours   : 0.1
Succeeded   : 16


**Would you exclude any samples? If yes, which and why?**

_SNI_Sal_2_ and _SNI_Sal_4_ show extreme differences to the rest of the samples, supported by the bad quality seen in the MultiQC file.

If rerunning the pipeline with those files excluded shows _Sham_oxy_1_ as an extreme outlier, that one should also be considered to be excluded.

**How many genes were differentially expressed in each contrast? Does this confirm what the paper mentions?**

```x

	                        	up	down
Sham_oxy versus Sham_Sal in condition 	8	0
SNI_Sal versus Sham_Sal in condition 	72	1
SNI_oxy versus Sham_Sal in condition 	1	1


The paper mentioned HDAC1/HDAC2. However the analysis with the pipeline shows a completely different result.
When comparing Sham_Sal versus SNI_oxy to see the impact of opioid withdrawl in SNI mice, CDr1 (linked to neural degeneration) was upregulated and Gm22614, a mRNA 5'-splice site recognition was down regulated. However since it seems to be required in long and complex pathways, no finite conclusion can be drawn.

The paper seems to be comparing enzymes/proteins instead of genes themselves, which makes direct comparison without further analysis challenging.

**The paper mentions differentially expressed genes in three brain regions : the NAc, mPFC and VTA. Briefly explain what these 3 regions are.**

```x
NAc: nucleus accumbens. Has a role in motivation, aversion, reward, addiction and a chronic pain states.

mPFC: medial prefrontal cortex . Guides control of cognitive actions. Top-down signals to other parts of the brain that guide the flow of activity along the pathways needed to perform a task.

VTA: ventral tegmental area. Houses dopamine neurons and is related to the reward system, motivation, cognition, and drug addiction


**Is there anyway from the paper and the material and methods for us to know which genes are included in these regions?**

```x
The paper does not mentione the genes in these regions directly, only the enzymes encoded by them, pathways and the amounts.

**Once you have your list of differentially expressed genes, do you think just communicating those to the biologists would be sufficient? What does the publication state?**

```x
Depending on the biologists background, I might have to first provide more information, for example via functional annotation. The paper states a change in the reward pathway, so pathways related to the genes would have to be found.


**Please reproduce the Venn Diagram from Figure 3, not taking into account the brain regions but just the contrasts mentionned.**

In [12]:
# Load the three datasets
file_a = "results_diff_ana_v3/tables/differential/a.deseq2.results.tsv"
file_b = "results_diff_ana_v3/tables/differential/b.deseq2.results.tsv"
file_c = "results_diff_ana_v3/tables/differential/c.deseq2.results.tsv"

import pandas as pd
from venn import venn
import matplotlib.pyplot as plt


# Read each file into a DataFrame
df_a = pd.read_csv(file_a, sep='\t')
df_b = pd.read_csv(file_b, sep='\t')
df_c = pd.read_csv(file_c, sep='\t')

# Define a function to filter for DEGs based on significance criteria
def filter_degs(df):
    return df[(df['padj'] < 0.05) & (df['log2FoldChange'].abs() > 1) & df['padj'].notnull()]['gene_id']

# Filter for significant DEGs
deg_a = set(filter_degs(df_a))  # DEGs for Sham-Oxy vs Sham-Sal
deg_b = set(filter_degs(df_b))  # DEGs for SNI-Sal vs Sham-Sal
deg_c = set(filter_degs(df_c))  # DEGs for SNI-Oxy vs Sham-Sal

# Create a dictionary for the Venn package
data_dict = {
    'Sham-Oxy vs Sham-Sal': deg_a,
    'SNI-Sal vs Sham-Sal': deg_b,
    'SNI-Oxy vs Sham-Sal': deg_c
}

# Generate the Venn diagram
venn(data_dict)

# Set the title and save the figure
output_filename = "filtered_gene_overlap_venn_diagram.png"
plt.title("Venn Diagram for DEGs Using `venn` Package")
plt.savefig(output_filename, format='png', dpi=300)
plt.close()

print(f"Venn diagram saved as '{output_filename}'.")



Venn diagram saved as 'filtered_gene_overlap_venn_diagram.png'.


![alt text](filtered_gene_overlap_venn_diagram.png "Title")